In [1]:
%run ../utilities/web_utilities.ipynb
%run ../utilities/database_utilities.ipynb
%run ../utilities/encoder_utilities.ipynb
%run ../utilities/pd_utilities.ipynb

In [2]:
write_mode = 'replace'
target_table = 'whl_player_game_stats'

In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../../')

import settings as st

In [5]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
db_host = st.database['local']['host']
db_port = st.database['local']['port']
db_user = st.database['local']['user']
db_pass = st.database['local']['pass']
db_sys = st.database['local']['system']
db_db = st.database['local']['db']

In [7]:
key = "41b145a848f4bd67"

In [8]:
sql = "SELECT DISTINCT player_id FROM whl_team_roster_by_season WHERE player_id IS NOT NULL ORDER BY player_id ASC"
res = query_database_to_list(db_sys, db_user, db_pass, db_host, db_port, db_db, sql)

In [9]:
master_df = pd.DataFrame()

In [12]:
i = 1
for r in res:
    player_id = str(r['player_id']).replace(".0", "")
    
    print(i, player_id)
    
    player_id = "27537"
    
    url = "http://lscluster.hockeytech.com/feed/?feed=modulekit&view=player&key={0}&fmt=json&client_code=whl&lang=en&player_id={1}&category=gamebygame".format(key, player_id)

    json_data = return_json(url)
    
    break
    





1 21812


In [13]:
json_data

{'SiteKit': {'Parameters': {'feed': 'modulekit',
   'view': 'player',
   'key': '41b145a848f4bd67',
   'fmt': 'json',
   'client_code': 'whl',
   'lang': 'en',
   'player_id': '27537',
   'category': 'gamebygame',
   'lang_id': 1,
   'league_id': '26',
   'season_id': '267'},
  'Player': {'seasons_played': [{'season_id': 267,
     'season_name': '2019 WHL Playoffs'},
    {'season_id': 266, 'season_name': '2018 - 19 Regular Season'},
    {'season_id': 265, 'season_name': '2018 Pre Season'},
    {'season_id': 264, 'season_name': '2018 WHL Playoffs'},
    {'season_id': 262, 'season_name': '2017 - 18 Regular Season'},
    {'season_id': 261, 'season_name': '2017 Pre Season'},
    {'season_id': 257, 'season_name': '2016 - 17 Regular Season'},
    {'season_id': 256, 'season_name': '2016 Pre Season'}],
   'games': [{'gMonth': 'March',
     'id': '1016384',
     'home_team': '215',
     'visiting_team': '208',
     'date_played': '2019-03-23',
     'home': '1',
     'shots': '2',
     'goalie':